In [5]:
import requests
import m3u8
import subprocess
import os
import sys
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import time
from tqdm import tqdm
from ulid import ULID

from moviepy.editor import VideoFileClip, concatenate_videoclips
import argparse

import ast
import uuid


# Get Longer Dataset

## WARNING: FILE WOULD BE EXTRA LARGE

In [8]:
m3u8_url = "https://cctvjss.jogjakota.go.id/malioboro/Malioboro_25_Utara_Mall.stream/chunklist_w1882981590.m3u8"
stream_url = 'https://cctvjss.jogjakota.go.id/malioboro/Malioboro_25_Utara_Mall.stream/'
save_directory = 'cctv-malioboro-download-v5'
os.makedirs(save_directory, exist_ok=True)
merge_directory = save_directory + '-merged'
os.makedirs(merge_directory , exist_ok=True)

ts_url_names = []
ts_urls = []

for _ in tqdm(range(300)):
    response = requests.get(m3u8_url, verify = False)
    time.sleep(2.5)
    m3u8_content = response.text
    m3u8_obj = m3u8.loads(m3u8_content)
    ts_url_name = [segment.uri for segment in m3u8_obj.segments]
    ts_url = [stream_url + x for x in ts_url_name]
    ts_url_names.extend(ts_url_name)
    ts_urls.extend(ts_url)


for i, (ts_url, ts_name) in tqdm(enumerate(zip(ts_urls, ts_url_names))):
    ts_response = requests.get(ts_url, verify = False)
    ts_filename = os.path.join(save_directory, f'{ts_name}')
    with open(ts_filename, 'wb') as ts_file:
        ts_file.write(ts_response.content)

ts_files = sorted([os.path.join(save_directory, f) for f in os.listdir(save_directory) if f.endswith('.ts')])

for ts_file in tqdm(ts_files, total = len(ts_files)):
    mp4_file = ts_file.replace('.ts', '.mp4')
    
    subprocess.run([
        'ffmpeg', '-i', ts_file, '-c', 'copy', mp4_file
    ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    os.remove(ts_file)

mp4_files = sorted([os.path.join(save_directory, f) for f in os.listdir(save_directory) if f.endswith('.mp4')])
first_fl = mp4_files[0].split('/')[-1].replace('.mp4', '')
last_fl = mp4_files[-1].split('/')[-1].replace('.mp4', '')

output_video = f"{first_fl}_{last_fl}.mp4"

clips = [VideoFileClip(file) for file in mp4_files]

final_clip = concatenate_videoclips(clips, method='compose')
for mpf in mp4_files:
    os.remove(mpf)
final_clip.write_videofile(os.path.join(merge_directory, output_video))



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:48<00:00,  1.09s/it]
135it [01:35,  1.41it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 18.20it/s]


Moviepy - Building video cctv-malioboro-download-v5-merged/media_w1882981590_308_media_w1882981590_308.mp4.
Moviepy - Writing video cctv-malioboro-download-v5-merged/media_w1882981590_308_media_w1882981590_308.mp4



Moviepy - Done !
Moviepy - video ready cctv-malioboro-download-v5-merged/media_w1882981590_308_media_w1882981590_308.mp4
